# Importing Libraries and Scripts

In [ ]:
from torch.utils.data import DataLoader
from learner import Learner
from loss import *
from dataset import *
import os
from sklearn import metrics
import numpy as np
import pickle

# Loading the dataset

In [ ]:

normal_train_dataset = Normal_Loader(is_train=1) # this is like a big container that has 810 videos in numpy array format, each of which has 32 segments. And each of this 32 segment is a image of 2048x2048
normal_test_dataset = Normal_Loader(is_train=0)
# print(Normal_Loader(is_train=1).data_list) # this contains listttssss ye babe ye you got this 
anomaly_train_dataset = Anomaly_Loader(is_train=1)
# print(len(anomaly_train_dataset))
anomaly_test_dataset = Anomaly_Loader(is_train=0)
# print(len(anomaly_test_dataset))
# print(normal_train_dataset)
normal_train_loader = DataLoader(normal_train_dataset, batch_size=30, shuffle=True)

normal_test_loader = DataLoader(normal_test_dataset, batch_size=1, shuffle=True)

anomaly_train_loader = DataLoader(anomaly_train_dataset, batch_size=30, shuffle=True) 
anomaly_test_loader = DataLoader(anomaly_test_dataset, batch_size=1, shuffle=True)

In [ ]:
list_of_test_video = []
path = '/home/dwip.dalal/shoplifting/Anomaly-Videos-Part-4/'
#path='/home/dwip.dalal/shoplifting_implementation_1/Real-world-Anomaly-Detection-in-Surveillance-Videos-pytorch/DATA/UCF-Crime/'
for i in anomaly_test_dataset.data_list:
    list_of_test_video.append(path+i.split('|')[0])
print(list_of_test_video[40])

In [ ]:
list_of_test_video[39]

### Algorithm to check number of frames in a video
Just to test for the difference in the number of frames in the video and the test dataset.

In [ ]:
import cv2
cap = cv2.VideoCapture(list_of_test_video[45])
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(length)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = Learner(input_dim=2048, drop_p=0.0).to(device)
optimizer = torch.optim.Adagrad(model.parameters(), lr= 0.001, weight_decay=0.0010000000474974513)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[25, 50])
criterion = MIL

In [ ]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (normal_inputs, anomaly_inputs) in enumerate(zip(normal_train_loader, anomaly_train_loader)):
        inputs = torch.cat([anomaly_inputs, normal_inputs], dim=1)
        batch_size = inputs.shape[0]
        inputs = inputs.view(-1, inputs.size(-1)).to(device)
        outputs = model(inputs)
        # print(outputs.shape)
        loss = criterion(outputs, batch_size)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print('loss = {}', train_loss/len(normal_train_loader))
    scheduler.step()

In [ ]:
def test_abnormal(epoch):
    model.eval()
    auc = 0
    with torch.no_grad():
        for i, (data, data2) in enumerate(zip(anomaly_test_loader, normal_test_loader)): # this make 140 iterations i.e. for 140 videos
            inputs, gts, frames = data 
            inputs = inputs.view(-1, inputs.size(-1)).to(torch.device('cuda'))
            # print(frames)
            score = model(inputs)
            # Score holds value corresponding each of the 32 segments of each video
            # MIL is calculated for each of the 32 segments of each video
            # so we shall need the score for each of the 32 segments of each video
            # this socre for 32 segemetns tells us the probabliy of anomaly happening in each of the 32 segments
            score = score.cpu().detach().numpy() 
            # print(score)
            score_list = np.zeros(frames[0]) # score_list will holds score corresponding to each of the frame in the entire video.
            
            step = np.round(np.linspace(0, frames[0]//16, 33))
            # print(step)
            # we have already computed C3D features for the whole video and divided the video features into 32 segments.

            for j in range(32): # 32 segment frames for one testing video
                score_list[int(step[j])*16:(int(step[j+1]))*16] = score[j]
            
           
            gt_list = np.zeros(frames[0]) # gt_list will hold the true value for each of frame in the video segement.
            for k in range(len(gts)//2):
                s = gts[k*2] # gts is the start and end of frames in which the anomaly will occur. 
                e = min(gts[k*2+1], frames)
                gt_list[s-1:e] = 1

            inputs2, gts2, frames2 = data2
            inputs2 = inputs2.view(-1, inputs2.size(-1)).to(torch.device('cuda'))
            score2 = model(inputs2)
            score2 = score2.cpu().detach().numpy()
            score_list2 = np.zeros(frames2[0])
            step2 = np.round(np.linspace(0, frames2[0]//16, 33))
            for kk in range(32):
                score_list2[int(step2[kk])*16:(int(step2[kk+1]))*16] = score2[kk]
            gt_list2 = np.zeros(frames2[0])
            score_list3 = np.concatenate((score_list, score_list2), axis=0)
            gt_list3 = np.concatenate((gt_list, gt_list2), axis=0)
            # print('this is gt_list', gt_list)
            # print('this is score_list3', score_list3) # this is the predicited binary score 1 for anomaly 0 for normal
            # so after looking at this two questions remain:
            # 1. what is the length of score_list3?
            # 2. what does 1 and 0 in it indicate?
            # print(len(gt_list3))
            fpr, tpr, thresholds = metrics.roc_curve(gt_list3, score_list3, pos_label=1)
            # print(fpr, tpr)
            if metrics.auc(fpr, tpr) != 'nan':
                auc += metrics.auc(fpr, tpr)
                
        print('auc is', auc/140)

In [ ]:
for epoch in range(0, 300):
    train(epoch)
    print(test_abnormal(epoch))

In [ ]:
import warnings

def fxn():
    print(test_abnormal(epoch))
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [ ]:
# model = pickle.load(open('moodelll.pkl', 'rb'))

In [ ]:
sass = [0]
with torch.no_grad():
        for i, (data, data2) in enumerate(zip(anomaly_test_loader, normal_test_loader)): # this make 140 iterations i.e. for 140 videos
                inputs, gts, frames = data 
                inputs = inputs.view(-1, inputs.size(-1)).to(torch.device('cuda'))
                # print(frames)
                score = model(inputs)
                # Score holds value corresponding each of the 32 segments of each video
                # MIL is calculated for each of the 32 segments of each video
                # so we shall need the score for each of the 32 segments of each video
                # this socre for 32 segemetns tells us the probabliy of anomaly happening in each of the 32 segments
                score = score.cpu().detach().numpy() 
                # print(score)
                score_list = np.zeros(frames[0]) # score_list will holds score corresponding to each of the frame in the entire video.
                
                step = np.round(np.linspace(0, frames[0]//16, 33))
                # print(step)
                # we have already computed C3D features for the whole video and divided the video features into 32 segments.

                for j in range(32): # 32 segment frames for one testing video
                    score_list[int(step[j])*16:(int(step[j+1]))*16] = score[j]
                if len(score_list) == 2804 or len(score_list) == 2803:
                    sass = score_list
for i in sass:
    print(i, end=' ')

In [ ]:
b = 35
list_of_test_video[b]

In [ ]:
# Function to see how is the performance of the model
a = 0
for i in sass:
    a = a + 1 
    if i>0.06:
        print(a, end=' ')
        
print(len(sass))

In [ ]:
%cd shoplifitng_implementation_1/
# %rm test_video_with_text

%mkdir test_video_with_text

In [ ]:
# pickle.dump(model, open('moodelll.pkl', 'wb')

In [ ]:
def draw_text(img, text,
          font=cv2.FONT_HERSHEY_SIMPLEX,
          pos=(0, 0),
          font_scale=1,
          font_thickness=1,
          text_color=(0, 0, 255),
          text_color_bg=(0, 0, 0)
          ):

    x, y = pos
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    cv2.rectangle(img, pos, (x + text_w, y + text_h), text_color_bg, -1)
    cv2.putText(img, text, (x, y + text_h + font_scale - 1), font, font_scale, text_color, font_thickness)

    return text_size

In [ ]:
# Python program to write
# text on video
import cv2
cap = cv2.VideoCapture(list_of_test_video[b])
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
a = 0

while(a<length):
    a = a + 1
    print(a)
    ret, frame = cap.read()

    font = cv2.FONT_HERSHEY_SIMPLEX

    if sass[a-1]>0.05: 
        draw_text(frame, "shoplifting", pos=(155, 30))
        # draw_text(frame, "shoplifting", font_scale=4, pos=(150, 50 + h), text_color_bg=(0, 0, 0))
        # cv2.putText(frame,'shoplifting',(155, 50),font, 1,(0, 0, 255),1,cv2.LINE_4)
    else:
        draw_text(frame, "normal", pos=(185, 30), text_color=(255, 255, 255))
        # cv2.putText(frame,'normal',(200, 50),font, 1,(255, 255, 255),2,cv2.LINE_4)

    path_save = '/home/dwip.dalal/shoplifting_implementation_1/Real-world-Anomaly-Detection-in-Surveillance-Videos-pytorch/test_video_with_text/' + str(a) + '.jpg'
    cv2.imwrite(path_save, frame)


In [ ]:
import cv2
import os

image_folder = '/home/dwip.dalal/shoplifting_implementation_1/Real-world-Anomaly-Detection-in-Surveillance-Videos-pytorch/test_video_with_text/'
video_name = '/home/dwip.dalal/shoplifting_implementation_1/Real-world-Anomaly-Detection-in-Surveillance-Videos-pytorch/video5.avi'

images = [int(img[:-4]) for img in os.listdir(image_folder)]
images = sorted(images)
images = [str(img) + '.jpg' for img in images]
print(images)
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 30, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()